In [4]:
%run '06_Complexes_Of_Equivariant_Vector_Bundles.ipynb'

In [5]:
class Lefschetz_Collection ( Algebraic_Structure ) :

    SUPPORT_PATTERN_LABELLING = { True  : [ True  , 1 , 'x' , 'X' ] ,
                                  False : [ False , 0 , 'o' , 'O' ]
                                }


    def __add__ ( self , other:"Lefschetz_Collection" ) -> "Lefschetz_Collection" :
        """Returns the concatenation of ``self`` with ``other``."""
        assert isinstance( other , self.__class__ ) , \
               TypeError('The input for ``other`` must be a Lefschetz collection.')
        assert self.Base_Space() == other.Base_Space() , \
               ValueError('The base spaces of ``self`` and ``other`` need to coincide.')
        assert self.Twist() == other.Twist() , \
               ValueError('The twists of ``self`` and ``other`` need to coincide.')

        New_Starting_Block = self.Starting_Block()+other.Starting_Block()

        New_Support_Pattern = dict({})
        Row_IDs = list(self.Support_Pattern().keys())+list(other.Support_Pattern().keys())
        if 0 < len(Row_IDs) :
            for Row_ID in [ min(Row_IDs) .. max(Row_IDs) ] :
                Row = []
                if Row_ID in self.Support_Pattern().keys() : Row += self.Support_Pattern()[Row_ID]
                else : Row += self.Width() * [ False ]
                if Row_ID in other.Support_Pattern().keys() : Row += other.Support_Pattern()[Row_ID]
                else : Row += other.Width() * [ False ]
                New_Support_Pattern.update({ Row_ID : Row })

        return self.__class__( Base_Space=self.Base_Space() , Starting_Block=New_Starting_Block , Twist=self.Twist() , Support_Pattern=New_Support_Pattern )


    def __call__ ( self , *Twists:tuple[int] ) -> "Lefschetz_Collection" :
        """Returns a Lefschetz collection with twisted starting block."""
        return self.__class__( Base_Space=self.Base_Space() , Starting_Block=[ Object(*Twists) for Object in self.Starting_Block() ] , Twist=self.Twist() , Support_Pattern=self.Support_Pattern() )


    def __getitem__ ( self , Input:int or slice
                    ) -> Irreducible_Equivariant_Vector_Bundle or Direct_Sum_Of_Equivariant_Vector_Bundles or Extension_Of_Equivariant_Vector_Bundles or \
                         list[ Irreducible_Equivariant_Vector_Bundle or Direct_Sum_Of_Equivariant_Vector_Bundles or Extension_Of_Equivariant_Vector_Bundles ] :
        """Returns a single datum or slices of data."""
        if Input in ZZ :
            Index = Input
            if   Index <= -len(self)-1 :         return None
            elif Index in [ -len(self) .. -1 ] : Index += len(self)
            elif Index in [ 0 .. len(self)-1 ] :   pass
            else :                               return None
            for Counter , Object in enumerate( self ) :
                if Counter == Index : return Object

        elif type(Input) == slice :
            Slice = Input
            Indices = range(len(self))[Slice]
            List = []
            for Counter , Object in enumerate( self ) :
                if Counter in Indices : List += [ Object ]
            return List

        else :
            raise ValueError('The variable ``Input`` is inappropriate.')


    def __init__ ( self , \
                   Base_Space:Irreducible_Homogeneous_Variety , \
                   Starting_Block:list[ Equivariant_Vector_Bundle ] , \
                   Twist:Irreducible_Equivariant_Vector_Bundle or None=None , \
                   Support_Pattern:list or str or None='Trivial'
                 ) -> None :
        """
        Initialize a Lefschetz collection over an irreducible homogeneous variety.

        INPUT:
        - ``Base_Space`` -- Irreducible Homogeneous Variety;
        - ``Starting_Block`` -- List;
        - ``Twist`` -- Vector bundle over an irreducible homogeneous variety;
        - ``Support_Pattern`` -- Tuple;

        OUTPUT: None.
        """
        # Base space
        assert isinstance( Base_Space , Irreducible_Homogeneous_Variety ) , \
               TypeError('The input for ``base space`` must be an irreducible homogeneous variety.')
        self._Base_Space = Base_Space

        # Starting block
        if isinstance( Starting_Block , tuple ) : Starting_Block = list(Starting_Block)
        self._Starting_Block = []
        if isinstance( Starting_Block , list ) :
            for Object_Counter , Object in enumerate( Starting_Block , start=1 ) :
                assert isinstance( Object , ( Irreducible_Equivariant_Vector_Bundle , Direct_Sum_Of_Equivariant_Vector_Bundles , Extension_Of_Equivariant_Vector_Bundles ) ) , \
                       'The '+str(Object_Counter)+'-th object need to be an equivariant vector bundle.'
                assert self._Base_Space == Object.Base_Space() , \
                       ValueError('The base space of the '+str(Object_Counter)+'-th object does not coincide with the given base space.')
                self._Starting_Block += [ Object ]
        else : raise ValueError( 'The input for ``Starting_Block`` is inappropriate.' )

        # Twist
        if Twist == None :
            if self._Base_Space.Parabolic_Subgroup().Is_Maximal() : Twist = self._Base_Space.calO(1)
            else : raise ValueError( 'For a non-maximal paralic subgroup, one need to clearify the twist explicitly.' )
        assert isinstance( Twist , Irreducible_Equivariant_Vector_Bundle ) , \
               TypeError('The input for ``Twist`` must be a vector bundle over an irreducible homogeneous variety.')
        assert Twist.Is_Line_Bundle() , \
               ValueError('The input for ``Twist`` need to be a line bundle.')
        self._Twist = Twist

        # Support pattern
        if Support_Pattern in [ 'Trivial' , None ] :
            self._Support_Pattern = { 0 : len(self._Starting_Block) * [ True ] }

        elif Support_Pattern in [ 'Maximal' , 'Max' , 'Maximal Length' , infinity ] :
            raise ValueError('The method constructing the maximal support pattern is yet not implemented.')

        elif isinstance( Support_Pattern , ( tuple , list ) ) :
            Support_Pattern = list(Support_Pattern)
            Cases = []
            for Entry_Counter , Entry in enumerate( Support_Pattern , start=1 ) :

                if   Entry in ZZ :
                    if not 0 <= Entry :
                        raise ValueError('If the '+str(Entry_Counter)+'-th entry of the input ``Support_Pattern`` is an integer, ' \
                                         + 'then it must be non-negative.')
                    if not Entry <= len(self._Starting_Block) :
                        raise ValueError('If the '+str(Entry_Counter)+'-th entry of the input ``Support_Pattern`` is an integer, ' \
                                         + 'then it must be bounded by the length of the starting block; namely <= '+str(len(self._Starting_Block))+'.'
                                        )
                    Label = 'Support partition'
                elif instance( Entry , [ tuple , list ] ) :
                    Entry = list(Entry)

                    for Subentry_Counter , Subentry in enumerate( Entry ) :
                        if not ( Subentry in self.SUPPORT_PATTERN_LABELLING[True] or Subentry in self.SUPPORT_PATTERN_LABELLING[False] ) :
                            raise ValueError('If the '+str(Entry_Counter)+'-th entry of the input ``Support_Pattern`` is a tuple or a list, ' \
                                             + 'then it must contain labellings '+str(self.SUPPORT_PATTERN_LABELLING[True])+' or '+str(self.SUPPORT_PATTERN_LABELLING[False])+'.'
                                            )
                    if not len(Entry) <= len(self._Starting_Block) :
                        raise ValueError('If the '+str(Entry_Counter)+'-th entry of the input ``Support_Pattern`` is an row with labellings, ' \
                                         + 'then its length must be bounded by the length of the starting block; namely <= '+str(len(self._Starting_Block))+'.'
                                        )
                    Label = 'Grid'

                if not Label in Cases : Cases += [ Label ]

            if   len( Cases ) == 0 : Case = 'Empty support pattern'
            elif len( Cases ) == 1 : Case = Cases[0]
            else : raise ValueError( 'The entries of the input ``Support_Pattern`` are inappropriate.' )

            if   Case == 'Empty support pattern' :
                self._Support_Pattern = dict({})
            elif Case == 'Support partition' :
                self._Support_Pattern = { Row_Counter : Row_Width*[ True ] + (len(self._Starting_Block)-Row_Width)*[ False ]
                                          for Row_Counter , Row_Width in enumerate( Support_Pattern )
                                        }
            elif Case == 'Grid' :
                self._Support_Pattern = { Row_Counter : [ Entry in self.SUPPORT_PATTERN_LABELLING[True] for Entry in Row ] + (len(self._Starting_Block)-len(Row))*[ False ]
                                          for Row_Counter , Row in enumerate( Support_Pattern )
                                        }

        elif isinstance( Support_Pattern , dict ) :
            self._Support_Pattern = dict({})
            for Row_ID , Row in Support_Pattern.items() :
                if not Row_ID in ZZ :
                    raise ValueError('If the input ``Support_Pattern`` is an dictionary, then the keys must be integers.')
                if not type( Row ) in [ list ] :
                    raise ValueError('If the input ``Support_Pattern`` is an dictionary, then the values must be lists.')
                if not len(Row) <= len(self._Starting_Block) :
                    raise ValueError('If the input ``Support_Pattern`` is an dictionary, ' \
                                     +'then the length of the rows must be bounded by the length of the starting block, namely <= '+str(len(self._Starting_Block))+'.')
                for Entry_Counter , Entry in enumerate( Row , start=1 ) :
                    if ( not Entry in self.SUPPORT_PATTERN_LABELLING[True] ) and ( not Entry in self.SUPPORT_PATTERN_LABELLING[False] ) :
                        raise ValueError('If the input ``Support_Pattern`` is an dictionary, then the '+str(Entry_Counter)+'-th entry in the row '+str(Row_ID) \
                                         + 'must be a labelling of the form '+str(self.SUPPORT_PATTERN_LABELLING[True])+' or '+str(self.SUPPORT_PATTERN_LABELLING[False])+'.'
                                        )
            for Row_ID , Row in Support_Pattern.items() :
                Adjusted_Row = [ Entry in self.SUPPORT_PATTERN_LABELLING[True] for Entry in Row ] + (len(self._Starting_Block)-len(Row))*[ False ]
                if True in Adjusted_Row : self._Support_Pattern.update({ Row_ID : Adjusted_Row })

            if 0 < len( self._Support_Pattern.keys() ) :
                for Row_ID in [ min(self._Support_Pattern.keys()) .. max(self._Support_Pattern.keys()) ] :
                    if not Row_ID in self._Support_Pattern.keys() :
                        self._Support_Pattern.update({ Row_ID : len(self._Starting_Block)*[ False ]  })

            Sorted_Row_IDs = sorted( self._Support_Pattern.keys() )
            self._Support_Pattern = { Row_ID : self._Support_Pattern[Row_ID] for Row_ID in Sorted_Row_IDs }

        else :
            raise TypeError( 'The input for ``Support_Pattern`` is inappropriate.' )

        # Gram matrix
        self._Gram_Matrix = dict({})


    def __iter__ ( self ) -> Iterator[ Equivariant_Vector_Bundle ] :
        """Returns a generator for the objects of ``self``."""
        for yPos , Row in self.Support_Pattern().items() :
            for xPos , Object_Is_Accessible in enumerate( Row ) :
                if Object_Is_Accessible :
                    yield self.Starting_Block(xPos)(yPos)


    def __len__ ( self ) -> int :
        """Returns the number of objects of ``self``."""
        return sum([ sum([ 1 for Object_Is_Accessible in Row if Object_Is_Accessible ]) for Row in self.Support_Pattern().values() ])


    def __lshift__ ( self , Shift:int=1 ) -> "Lefschetz_Collection" :
        """Returns the left shift of ``self``."""
        return self >> -Shift


    def __repr__ ( self ) :
        """Returns all attributes which are necessary to initialize the object."""
        return self.Base_Space() , self.Starting_Block() , self.Twist() , self.Support_Pattern()


    def __rshift__ ( self , Shift:int=1 ) -> "Lefschetz_Collection" :
        """Returns a sheft of ``self``."""
        assert Shift in ZZ , \
               TypeError('The input for ``Shift`` need to be an integer.')
        New_Support_Pattern = { Row_ID+Shift : Row for Row_ID , Row in self.Support_Pattern().items() }
        return self.__class__( Base_Space=self.Base_Space() , Starting_Block=self.Starting_Block() , Twist=self.Twist() , Support_Pattern=New_Support_Pattern )(-Shift)


    def __str__ ( self ) -> str :
        """Returns a one-line string as short description."""
        return 'Lefschetz collection consisting of '+str(len(self))+' objects in a grid with width = '+str(self.Width())+' and height = '+str(self.Height())


    def Base_Space ( self ) -> Irreducible_Homogeneous_Variety :
        """Returns the attribute ``_Base_Space``."""
        return self._Base_Space


    # Synonym for the method ``Row`` :
    def Block ( self , Input:int ) -> Iterator[ Equivariant_Vector_Bundle ] :
        """Returns a single block (= row)."""
        self.Row( Input )


    def Column ( self , Input:int ) -> Iterator[ Equivariant_Vector_Bundle ] :
        """Returns a single column."""
        Width = self.Width()
        if   Input in ZZ :
            if   Input <= -Width-1 :
                Start_Iteration = False
            elif Input in [ -Width .. -1 ] :
                Start_Iteration = True
                xPos = Input + Width
            elif Input in [ 0 .. Width-1 ] :
                Start_Iteration = True
                xPos = Input
            elif Width <= Input :
                Start_Iteration = False

            if Start_Iteration :
                for yPos , Row in self.Support_Pattern().items() :
                    Object_Is_Accessible = Row[xPos]
                    if Object_Is_Accessible : yield self.Starting_Block(xPos)(yPos)

        else :
            raise TypeError('The input data must be an integer in the range '+str( [ -Width .. Width-1 ] )+'.')


    def Gram_Matrix ( self , Base_Ring:'Ring'=ZZ , Compute_Now:bool=False ) -> Matrix :
        """
        Returns a Gram matrix associated to the computations of EXT( E_p , E_q ) (p: row , q: column).
        For the entry in the p-th row and q-th column, it is the dim Euler_Sum( E_p , E_q ) = dim ( sum_i (-1)^i EXT^i( E_p , E_q ) ).
        """
        if Compute_Now == True :
            WCR = self.Base_Space().Parent_Group().Weyl_Character_Ring()
            if   Base_Ring == ZZ  : self._Gram_Matrix.update({ ZZ  : Matrix( ZZ  , [ [ E1.Euler_Sum(E2).degree() for E2 in self ] for E1 in self ] ) })
            elif Base_Ring == WCR : self._Gram_Matrix.update({ WCR :               [ [ E1.Euler_Sum(E2)          for E2 in self ] for E1 in self ]   })
            else                  : raise ValueError("The input for `Base_Ring` is inappropriate.")

            return self._Gram_Matrix[Base_Ring]

        else :
            if Base_Ring in self._Gram_Matrix.keys() : return self._Gram_Matrix[Base_Ring]
            else : return self.Gram_Matrix( Base_Ring=Base_Ring , Compute_Now=True )


    def Grid ( self , Labelling:list or None=None ) -> list[ list[ str ] ] :
        """Returns a grid of the objects of ``self``."""
        if Labelling == None : Labelling = [ 'E'+str(xPos) for xPos in range(self.Width()) ]
        assert type(Labelling) == list , TypeError('The input for ``Labelling`` must be a list.')
        assert len(Labelling) == self.Width() , ValueError('The ``Labelling`` must be of length '+str(len(self.Width()))+'.')
        for Label_Counter , Label in enumerate( Labelling , start=1 ) :
            assert type(Label) == str , TypeError('The '+str(Label_Counter)+'-th label must be a string.')

        Table = []
        for yPos , self_Row in self.Support_Pattern().items() :
            Table_Row = []
            for xPos , Object_Is_Accessible in enumerate( self_Row ) :
                if Object_Is_Accessible :
                    Label = Labelling[xPos]
                    if not yPos == 0: Twist = '('+str(yPos)+')'
                    else : Twist = ''
                    Table_Row_Entry = Label+Twist
                else : Table_Row_Entry = ''
                Table_Row += [ Table_Row_Entry ]
            Table += [ Table_Row ]

        if 0 < len(self) : return table( Table )


    def Has_Maximal_Expected_Length ( self ) -> bool :
        """Tests if ``self`` is maximal; i.e. len(self) == rk K_0(X)."""
        return len(self) == self.Base_Space().Grothendieck_Group().rank()


    def Height ( self ) -> int :
        """Returns the height of ``self``."""
        return len(self.Support_Pattern().keys())


    def Is_Exceptional ( self , Test_Numerically:bool=False ) -> bool :
        """
        Tests if the Lefschetz collection is (numerically) exceptional.

        INPUT:
        - ``self`` -- LefschetzCollection.

        OUTPUT:
        - Boolean; return True if ``self`` is exceptional, otherwise False as soon as possible.

        ALGORITHM:
        Compare to Lemma 2.2. in [Kuz2006]_

        REFERENCE:
        - [Kuz2006] Alexander Kuznetsov: Exceptional collections for Grassmannians of isotropic lines; 2006.
        """
        assert type(Test_Numerically) == bool , \
               TypeError('The input for ``Test_Numerically`` need to be boolean.')

        # Test if the accessible objects are exceptional.
        # Let E be from the starting block.
        # If E is accessible in the same row (i.e. E(y) is object of ``self`` for some y), then test if E is exceptional.
        # NOTE: EXT( E(y) , E(y) ) = EXT( E , E )
        for xPos in range(self.Width()) :
            # Test if the object is accessible in some row.
            Object_Is_Accessible_In_Same_Row = False
            for Row in self.Support_Pattern().values() :
                if Row[xPos] == True : Object_Is_Accessible_In_Same_Row = True ; break
            # If the object is accessible in some row, then test if it is exceptional.
            if Object_Is_Accessible_In_Same_Row :
                Obj = self.Starting_Block(xPos)
                if not Obj.Is_Exceptional(Test_Numerically) : return False

        # Test for accessible objects of the same row, if latter ones are semi-orthogonal to previous ones.
        # Let E_1 , E_2 be from the starting block and E_1 is positioned before E_2.
        # If E_1 and E_2 are simultaneously accessible in the same row (i.e. E_1(y) and E_2(y) are both objects of ``self`` for some y), then test if E_2 is semi-orthogonal to E_1.
        # NOTE: EXT( E_2(y) , E_1(y) ) = EXT( E_2 , E_1 )
        for xPos1 in range(self.Width()) :
            for xPos2 in range(self.Width())[ xPos1+1 : ] :
                # Test if there is a row where both objects are simultaneously accessible.
                Both_Objects_Are_Simultaneously_Accessible_In_The_Same_Row = False
                for Row in self.Support_Pattern().values() :
                    if Row[xPos1] == True and Row[xPos2] == True : Both_Objects_Are_Simultaneously_Accessible_In_The_Same_Row = True ; break
                # If both objects are simultaneously accessible in the same row, then check semi-orthogonality.
                if Both_Objects_Are_Simultaneously_Accessible_In_The_Same_Row :
                    Obj1 = self.Starting_Block(xPos1)
                    Obj2 = self.Starting_Block(xPos2)
                    for Obj2_Is_SemiOrthogonal_To_Obj1 in Obj2.Is_SemiOrthogonal_To( Obj1 , Test_Numerically ) :
                        if not Obj2_Is_SemiOrthogonal_To_Obj1 : return False

        # Test for accessible objects of the same row, if latter ones are semi-orthogonal to previous ones.
        # Let E_1 , E_2 be from the starting block.
        # If E_1 and E_2 are simultaneously accessible with respect to the row-difference yDelta (i.e. E_1(y_1) and E_2(y_2) are both objects of ``self``
        # for some y_1 and y_2 with yDela = y_2-y_1), then test if E_2(yDelta) is semi-orthogonal to E_1.
        # NOTE: EXT( E_2(y_2) , E_1(y_1) ) = EXT( E_2(yDelta) , E_1 )
        for xPos1 in range(self.Width()) :
            for xPos2 in range(self.Width()) :
                for yDelta in range(self.Height())[ 1 : ] :
                    # Test if there are two rows with yDelta difference where both objects are simultaneously accessible.
                    Both_Objects_Are_Simultaneously_Accessible_In_Rows_With_Difference_yDelta = False
                    for Row1_ID in self.Support_Pattern().keys() :
                        Row2_ID = Row1_ID + yDelta
                        if not Row2_ID in self.Support_Pattern().keys() : break
                        Row1 = self.Support_Pattern()[Row1_ID]
                        Row2 = self.Support_Pattern()[Row2_ID]
                        if Row1[xPos1] == True and Row2[xPos2] == True : Both_Objects_Are_Simultaneously_Accessible_In_Rows_With_Difference_yDelta = True ; break

                    # If both objects are simultaneously accessible with respect to yDelta, then check semi-orthogonality.
                    if Both_Objects_Are_Simultaneously_Accessible_In_Rows_With_Difference_yDelta :
                        Obj1 = self.Starting_Block(xPos1)
                        Obj2 = self.Starting_Block(xPos2) * self.Twist()^yDelta
                        for Obj2_Is_SemiOrthogonal_To_Obj1 in Obj2.Is_SemiOrthogonal_To( Obj1 , Test_Numerically ) :
                            if not Obj2_Is_SemiOrthogonal_To_Obj1 : return False

        if 0 < len(self) : return True


    def Is_Numerically_Exceptional ( self ) -> bool :
        """Tests if the Lefschetz collection is numerically exceptional."""
        return self.Is_Exceptional( Test_Numerically=True )


    def Numerical_Left_Dual ( self , Index:int ) -> Equivariant_Vector_Bundle :
        """Returns the i-th numerical left dual with respect to ``self.``"""
        l = len(self)-1
        assert Index in [ 0 .. l ] , \
               'The input for ``Index`` need to be in the range from 0 to '+str(l)+'.'
        LD = LC[l-Index]
        for j in range(l-Index+1,l+1) :
            LD = self[j].Numerical_Right_Mutation( LD )
        return LD


    def Numerical_Right_Dual ( self , Index:int ) -> Equivariant_Vector_Bundle :
        """Returns the i-th numerical right dual with respect to ``self.``"""
        l = len(self)-1
        assert Index in [ 0 .. l ] , \
               'The input for ``Index`` need to be in the range from 0 to '+str(l)+'.'
        RD = LC[l-Index]
        for j in range(l-Index-1,0,-1) :
            RD = self[j].Numerical_Left_Mutation( RD )
        return RD


    # Synonyms for the method ``Column`` :
    def Orbit ( self , Input:int )  -> Iterator[ Equivariant_Vector_Bundle ] :
        """Returns a single orbit (= column)."""
        self.Column( Input )


    def Row ( self , Input:int ) -> Iterator[ Equivariant_Vector_Bundle ] :
        """Returns a single row."""
        Height = self.Height()
        if   Input in ZZ :
            if   Input <= -Height-1 :
                Start_Iteration = False
            elif Input in [ -Height .. -1 ] :
                Start_Iteration = True
                yPos = Input + Height
            elif Input in [ 0 .. Height-1 ] :
                Start_Iteration = True
                yPos = Input
            elif Height <= Input :
                Start_Iteration = False

            if Start_Iteration :
                for xPos , Object_Is_Accessible in enumerate( self.Support_Pattern()[yPos] ) :
                    if Object_Is_Accessible : yield self.Starting_Block(xPos)(yPos)

        else :
            raise TypeError('The input data must be an integer in the range '+str( [ -Height .. Height-1 ] )+'.')


    def Subcollection( self , Columns:list or None=None , Rows:list or None=None ) -> "Lefschetz_Collection" :
        """Returns the subcollection of the given columns and rows."""
        if Columns == None : Columns = range( len( self.Starting_Block() ) )
        assert type(Columns) in [ list , range ] , \
               TypeError('The input for ``Columns`` need to be a list or a range.')
        if Rows    == None : Rows = list( self.Support_Pattern().keys() )
        assert type(Rows) in [ list , range ] , \
               TypeError('The input for ``Rows`` need to be a list or a range.')

        New_Starting_Block = [ Object for Object_Counter , Object in enumerate( self.Starting_Block() ) if Object_Counter in Columns ]
        New_Support_Pattern = { Row_ID : [ Value for Column_Counter , Value in enumerate( Row ) if Column_Counter in Columns ] \
                                for Row_ID , Row in self.Support_Pattern().items() \
                                if Row_ID in Rows
                              }
        return self.__class__( Base_Space=self.Base_Space() , Starting_Block=New_Starting_Block , Twist=self.Twist() , Support_Pattern=New_Support_Pattern )


    def Support_Pattern ( self ) -> list[ list[ bool ] ] :
        """Returns the attribute ``_Support_Pattern``."""
        return self._Support_Pattern


    def Starting_Block ( self , Input:None or int or slice=None ) -> Equivariant_Vector_Bundle or list[ Equivariant_Vector_Bundle ] :
        """Returns the attribute ``_Starting_Block``."""
        if   Input == None : return self._Starting_Block
        elif Input in ZZ :
            Index = Input % len(self._Starting_Block)
            return self._Starting_Block[Index]
        elif type(Input) == slice :
            Slice = Input
            return self._Starting_Block[Slice]
        else : raise ValueError('The input data is inappropriate.')


    def Twist ( self ) -> Irreducible_Equivariant_Vector_Bundle :
        """Returns the attribute ``_Twist``."""
        return self._Twist


    def Width ( self ) -> len :
        """Returns the width of ``self``."""
        return len(self.Starting_Block())


    def Mutate ( self , Dictionary:dict , Base_Ring:'Ring'=ZZ , Basis=None , Gram_Matrix=None ) -> ( list[ list[ 'Ring element' ] ] , list[ list[ 'Ring element' ] ] ) :
        """Describes the left mutation on the K_0-level."""
        # ..ToDo: Implement test for thoses objects, which are mutated through, are exceptional.

        Length = len(self)

        WCR = self.Base_Space().Parent_Group().Weyl_Character_Ring()
        if   Base_Ring == ZZ  : pass
        elif Base_Ring == WCR : pass
        else                  : raise ValueError('The input for ``Base_Ring`` is inappropriate.')

        if Basis == None : Basis = [ i*[ Base_Ring(0) ] + [ Base_Ring(1) ] + (Length-1-i)*[ Base_Ring(0) ] for i in range(Length) ]

        assert type(Basis) == list , \
               TypeError('The input for ``Basis`` must be a list.')
        for Element_Counter , Element in enumerate( Basis , start=1 ) :
            assert type(Element) == list , \
                   ValueError('The '+str(Element_Counter)+'-th element is not a list.')
            for Entry_Counter , Entry in enumerate( Element , start=1 ) :
                assert Entry in Base_Ring , \
                       ValueError('The '+str(Entry_Counter)+'-th entry of the '+str(Element_Counter)+'-th element is not an object of the base ring, namely '+str(Base_Ring)+'.')

        if Gram_Matrix == None : Gram_Matrix = self.Gram_Matrix( Base_Ring )

        for Row_Counter , Row in enumerate( Gram_Matrix , start=1 ) :
            for Column_Counter , Entry in enumerate( Row , start=1 ) :
                assert Entry in Base_Ring , \
                       ValueError('The '+str(Column_Counter)+'-th entry of the '+str(Row_Counter)+'-th row is not an object of the base ring, namely '+str(Base_Ring)+'.')
            assert Column_Counter == Length , \
                   ValueError('The Gram matrix does not have rows of length '+str(Length)+'.')
        assert Row_Counter == Length , \
            ValueError('The Gram matrix does not have columns of length '+str(Length)+'.')

        Bases = dict({})
        Gram_Matrices = dict({})

        Timer = 0
        Bases.update({ Timer : Basis })
        Gram_Matrices.update({ Timer : Gram_Matrix })

        for Mutation_Counter , ( Start , End ) in enumerate( Dictionary.items() , start=1 ) :
            assert Start in range(Length) , \
                   ValueError('The '+str(Mutation_Counter)+'-th key of the mutation dictionary is not an element in the range '+str([ 0 .. Length-1 ])+'.')
            assert End in range(Length) , \
                   ValueError('The '+str(Mutation_Counter)+'-th value of the mutation dictionary is not an element in the range '+str([ 0 .. Length-1 ])+'.')

            Index = Start
            while Index != End :
                Timer += 1
                if End <= Start : # Left mutation
                    # Mutation triangle: 0 ----> EXT( E , F ) * E --ev--> F ----> LL_E(F) ----> 0
                    # i.e. [ LL_E(F) ] = [ F ] - EXT( E , F ) * [ E ]  in K_0( self.Base_Space() )
                    e = Bases[Timer-1][ Index-1 ] # The object which is mutated through
                    f = Bases[Timer-1][ Index   ] # The object which is mutated
                    chi = Gram_Matrices[Timer-1][ Index-1 ][ Index ]
                    mutation = [ f[i] - chi * e[i] for i in range(Length) ]

                    Bases.update({ Timer : Bases[Timer-1][ : Index-1 ] + [ mutation  , e ] + Bases[Timer-1][ Index+1 : ] })
                    Gram_Matrices.update({ Timer : [ [ sum([ Base_Ring(0) ] + [ Bases[Timer][ Row_Index ][j] * Gram_Matrix[j][i] * Bases[Timer][ Column_Index ][i]
                                                                                for i in range(Length)
                                                                                for j in range(Length)
                                                                              ])
                                                       for Column_Index in range(Length)
                                                     ]
                                                     for Row_Index in range(Length)
                                                   ]
                                        })
                    Index -= 1

                else : # Right mutation
                    # Mutation triangle: 0 ----> RR_F(E) ----> E --coev--> EXT( F , E )^vee * F ----> 0    ???
                    # i.e. [ RR_F(E) ] = ?
                    #e = ? # The object which is mutated
                    #f = ? # The object which is mutated through
                    #chi = ?
                    #mutation = ?

                    Bases.update({ Timer : Bases[Timer-1][ : Index-1 ] + [ f , mutation ] + Bases[Timer-1][ Index+1 : ] })
                    #Gram_Matrices.update({ Timer : ? })

                    raise ValueError('The method for right mutations is yet not implemented.')
                    Index += 1

        return Bases[Timer] , Gram_Matrices[Timer]


    def Test_For_Numerically_Exceptional_Proper_Extension ( self ,
                                                            New_Object :Equivariant_Vector_Bundle , Relevant_Columns :set[ int ] or None =None ,
                                                            Test_If_Self_Is_Exceptional :bool =True
                                                          ) -> Iterator[ int ] :
        Test_Numerically=True

        assert isinstance( New_Object , Equivariant_Vector_Bundle ) , \
               'The input for `New_Object` need to be an equivariant vector bundle.'
        New_Orbit = New_Object.Maximal_Numerically_Exceptional_Orbit()

        if Relevant_Columns == None : Relevant_Columns = set(range(self.Width()+1))
        assert isinstance( Relevant_Columns , set ) , \
               'If the input for `Relevant_Columns` is not None, then it need to be a set.'
        assert Relevant_Columns.issubset( set(range(self.Width()+1)) ) , \
               'The relevant columns need to be in the range from 0 to '+str(self.Width()+1)+'.'

        if Test_If_Self_Is_Exceptional == True :
            assert self.Is_Exceptional( Test_Numerically=Test_Numerically ) , \
                   'The collection `self` is not exceptional.'

        def Test_For ( xPos1 , yPos1 ) :
            Object1 = New_Object(yPos1)
            for yPos2 , Row in self.Support_Pattern().items() :
                for xPos2 , Object2_Is_Accessible in enumerate( Row ) :
                    if Object2_Is_Accessible :
                        Object2 = self.Starting_Block(xPos2)(yPos2)
                        #if not xPos2 in Already_Tested.keys() : Already_Tested.update({ xPos2 : ( set({}) , set({}) ) })
                        if yPos2 < yPos1 or ( yPos2 == yPos1 and xPos2 < xPos1 ) :
                            #if not yPos1-yPos2 in Already_Tested[xPos2][0] :
                            #    Already_Tested[xPos2][0].add( yPos1-yPos2 )
                            if not next( Object1.Is_SemiOrthogonal_To( Object2 , Test_Numerically=Test_Numerically ) ) : return False
                        else                                                     :
                            #if not yPos2-yPos1 in Already_Tested[xPos2][1] :
                            #    Already_Tested[xPos2][1].add( yPos2-yPos1 )
                            if not next( Object2.Is_SemiOrthogonal_To( Object1 , Test_Numerically=Test_Numerically ) ) : return False
            return True

        for Tested_xPos in Relevant_Columns :
            #Already_Tested = dict({})
            yield ( Tested_xPos , [ Tested_yPos for Tested_yPos in range(len(New_Orbit)) if Test_For( Tested_xPos , Tested_yPos ) ] )

In [78]:
#X = Orthogonal_Grassmannian(3,9)
#fw = X.Basis( 'fw' )
#LC  = X.calO().Maximal_Numerically_Exceptional_Orbit()
#LC += X.calU( fw[1] ).Maximal_Numerically_Exceptional_Orbit()
#LC += X.calU( fw[2] ).Maximal_Numerically_Exceptional_Orbit()
#
#New_Object = X.calU( 2*fw[1]+fw[4] ).Extend_Equivariantly_By( X.calU( fw[1]+fw[4] ) )
#for Result in LC.Test_For_Numerically_Exceptional_Proper_Extension( New_Object=New_Object ) :
#    print(Result)